In [ ]:
%%capture

!pip install gdown

Analyze screen time on MacOS to see where I spend my time:

```
cd ~/gitRepo/ScreenTime2CSV/ScreenTime2CSV
python3 screentime2csv.py -o output.tsv -d '\t'
```

In [ ]:
%%capture

import gdown

URL = 'https://drive.google.com/file/d/1aXTzmjvdvLesw1MH26wAL2UGC1EjL0sn/view?usp=sharing'
gdown.download(URL, 'output.tsv', quiet=True, fuzzy=True)

In [ ]:
import pandas as pd

df = pd.read_csv('output.tsv', sep='\t')
df.head(3)

In [ ]:
df['date'] = pd.to_datetime(df['created_at'], unit='s').dt.strftime('%d-%m-%Y')
df['datetime'] = pd.to_datetime(df['created_at'], unit='s', utc=True).dt.tz_convert('Asia/Ho_Chi_Minh')

df = df.sort_values(by='datetime', ascending=False)
df = df[df['datetime'].dt.weekday < 5]

df.head(3)

In [ ]:
daily_usage = df.groupby(['date'])['usage'].sum().reset_index()

daily_usage['hours'] = (daily_usage['usage'] // 3600).astype(int)
daily_usage['minutes'] = ((daily_usage['usage'] % 3600) // 60).astype(int)

daily_usage['datetime'] = pd.to_datetime(df['date'], format="%d-%m-%Y")
daily_usage['day_of_week'] = daily_usage['datetime'].dt.strftime('%A')

daily_usage['sort_date'] = pd.to_datetime(daily_usage['date'], format='%d-%m-%Y')
daily_usage = daily_usage.sort_values(by='sort_date', ascending=False)

median_usage = daily_usage['usage'].median()

median_hours = int(median_usage // 3600)
median_minutes = int((median_usage % 3600) // 60)

print(f"\nMedian: {median_hours}h:{median_minutes}m\n")

for index, row in daily_usage.iterrows():
    print(f"{row['date']}: {row['hours']}h:{row['minutes']}m")

In [ ]:
import pytz

df = df.sort_values(by=['date', 'datetime'])

# Compute time difference from previous entry
df['prev_time'] = df.groupby('date')['datetime'].shift(1)
df['gap_minutes'] = (df['datetime'] - df['prev_time']).dt.total_seconds() / 60

# Identify activity blocks
df['block'] = (df['gap_minutes'] >= 30).cumsum()  # New block starts if gap ≥ 30 minutes

# Compute start and end time for each block
activity_blocks = df.groupby(['date', 'block']).agg(
    start_time=('datetime', 'first'),
    end_time=('datetime', 'last')
).reset_index()

# Compute duration of each block
activity_blocks['duration_minutes'] = (activity_blocks['end_time'] - activity_blocks['start_time']).dt.total_seconds() / 60

# Keep only blocks that last at least 30 minutes
valid_blocks = activity_blocks[activity_blocks['duration_minutes'] >= 30]

for date, group in valid_blocks.groupby('date'):
    print(f"\nDate: {date}")
    for _, row in group.iterrows():
        start = row['start_time'].strftime('%H:%M')
        end = row['end_time'].strftime('%H:%M')
        duration = int(row['duration_minutes'])
        print(f"  - {start} to {end} ({duration} min)")
